# Support Vector Machines

````{margin}
```{note}
You may note that support vector machines also suffer from problems regarding non-linearity. 
But this approach takes advantage of linear algebra to reconfigure the space if necessary. 
```
````
An alternative classification to logistic regression is support vector machines. 
The aim of the support vector machine (SVM) is to find the best linear *hyperplane* that separates the classes in feature space. 
This is achieved by maximising the, so-called, margin, which is the distance between the hyperplane and the nearest data points from each class. 
These data points are the support vectors. 

```{figure} ../images/svm.png
---
name: svm
width: 40%
---
The margin (represented with the red shaded area) describes the region that separates the yellow and blue support vectors (thick black edge). 
```

Similar to logistic regression, the hyperplane that an SVM tries to find has the equation, 

$$
\mathbf{w}\cdot\mathbf{x} + b = 0.
$$

The two hyperplanes that describe the edges of the margin can be written as

$$
\mathbf{w}\cdot\mathbf{x} + b = 1,
$$

where the items are of class 1, and 

$$
\mathbf{w}\cdot\mathbf{x} + b = -1,
$$

for class -1. 
````{margin}
```{note}
See the following [math.stackexchange discussion](https://math.stackexchange.com/questions/1305925/why-is-the-svm-margin-equal-to-frac2-mathbfw) to see why the distance is $\frac{2}{|\mathbf{w}|}$. 
```
````
The distance between these two hyperplanes is $\frac{2}{|w|}$, so by minimising $|\mathbf{w}|$, the maximise this distance.\
However, is it also necessary to keep data points on their correct side of the margin, which is achieved with an appropriate constraint. 

## Soft Margin

It may be impossible to find a hyperplane that completely separates the classes. 
For example, consider the data from the [toy example for logistic regression](./logistic-regression.html).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("../data/toy.csv")

fix, ax = plt.subplots()
sns.scatterplot(x='x', y='y', data=data, hue='label', ax=ax)
plt.show()

It it clear that due to the overlap, there is no hyperplane possible. 
SVMs handle this through the inclusion in of what is known as a *soft margin*. 
The soft margin introduces some allowance for misclassification of data points. 
This leads the loss function to be the **hinge loss**, 

$$
L = \frac{1}{2}|\mathbf{w}|^2 + C\sum_{i=1}^{n}\textrm{max}(0, 1 - y_i(\mathbf{w}\cdot \mathbf{x}_i + b)),
$$

where $C$ is the hyperparameter that controls the trade off between maximising the margin and minimising misclassification.

## Putting It Into Practice

Let's see if we can develop a support vector machine to classify our toy data. 
Once again, we will split the data, we delibrately split it the same way as previously, to enable comparison, and create separate objects for the training dataset. 

In [2]:
from sklearn.model_selection import train_test_split

data['encoded_label'] = [1 if i == 'b' else 0 for i in data['label'].values]
train, test = train_test_split(data, test_size=0.2, random_state=0)
X = train[['x', 'y']].values
y = train['encoded_label'].values


Again, we generate our initial weights and bias. 

In [ ]:
import numpy as np

rng = np.random.default_rng(1)
weights = rng.standard_normal(2)
bias = rng.standard_normal(1)
weights, bias

The next step is to define the loss function, learning rate and $C$. 
Again, we use JAX here, to help our gradient descent algorithm. 

In [4]:
import jax.numpy as jnp
from jax import grad

C = 0.1
learning_rate = 0.001

def loss_function(weights, bias):
    prediction = y * (jnp.dot(X, weights) + bias)
    stack = jnp.hstack([jnp.zeros_like(y), 1 - prediction])
    soft_margin = C * jnp.mean(jnp.max(stack, axis=0))
    return 0.5 * jnp.linalg.norm(weights) ** 2 + soft_margin

first_order = grad(loss_function)

We now run 2000 iterations of the gradient descent algorithm. 

In [5]:
result = grad(loss_function)(weights, bias)

for i in range(2000):
    weights = weights - learning_rate * result[:-1]
    bias = bias - learning_rate * result[-1]
    result = grad(loss_function)(weights, bias)

As before, we compute the accuracy score against the training and test data. 

In [ ]:
from sklearn.metrics import accuracy_score

z = np.dot(train[['x', 'y']], weights) + bias
f = np.sign(z)
prediction = np.array(['nd'] * len(z))
prediction[np.where(f > 0)] = 'b'
prediction[np.where(f < 0)] = 'a'
train['prediction'] = prediction
z
accuracy_score(train['label'], train['prediction'])

In [ ]:
z = np.dot(test[['x', 'y']], weights) + bias
f = np.sign(z)
prediction = np.array(['x'] * len(f))
prediction[np.where(f > 0)] = 'b'
prediction[np.where(f < 0)] = 'a'
test['prediction'] = prediction

accuracy_score(test['label'], test['prediction'])

This gives similar, but slightly different, values to the logistic regression.
This is to be expected, as we are optimising different functions in order to enable the discrimiation. 

Thus far, both of these methods are limited to data with a linear discrimination. 
Let's look now at a tool employed by support vector machines to help in this area. 